In [ ]:
import os, json
from brownie import Contract, network, accounts

#### connect to avalanche network

In [ ]:
network_name = 'avax-main'
network.connect(network_name)

#### load usdc and aave pool contracts

In [ ]:
usdc = Contract.from_explorer('0xB97EF9Ef8734C71904D8002F8b6Bc66Dd9c48a6E')
usdc

using usdc erc20 abi for aave deposit token since only need this to call balance

In [ ]:
aAvaUSDC = Contract.from_abi('aAvaUSDC', '0x625E7708f30cA75bfd92586e17077590C60eb4cD', usdc.abi)

aave V3 contracts not verified and ABI not avaiable on block explorer so pulled abi from npm package: https://github.com/aave/aave-v3-core#getting-started

node:
```
const PoolV3Artifact = require('@aave/core-v3/artifacts/contracts/protocol/pool/Pool.sol/Pool.json');
const abi = JSON.stringify(PoolV3Artifact.abi)

const fs = require('fs');
fs.writeFile("aave_v3_pool_abi.json", abi, 'utf8', function (err) {
    if (err) {
        console.log("An error occured while writing JSON Object to File.");
        return console.log(err);
    }

    console.log("JSON file has been saved.");
});
```

In [ ]:
with open('aave_v3_pool_abi.json', 'r') as f:
    abi = json.load(f)

aave_pool = Contract.from_abi('Aave V3 Pool', '0x794a61358d6845594f94dc1db02a252b5b4814ad', abi)

#### load account  

In [ ]:
acc = accounts.add(os.environ['ACC_PRIVATE_KEY'])

# TO DO: replace `acc` with fireblocks connection logic

In [ ]:
acc = 

#### check account balances

In [ ]:
('AVAX: {:.02f}, USDC: {:.02f}, aAvaUSDC: {:.02f}'
 .format(acc.balance() / (10 ** 18), 
         usdc.balanceOf(acc.address) / (10 ** usdc.decimals()),
         aAvaUSDC.balanceOf(acc.address) / (10 ** aAvaUSDC.decimals())))

#### deposit $1

In [ ]:
txn = aave_pool.deposit(usdc.address, 1000000, acc.address, 0, {'from': acc})
txn.wait(7)

#### check account balances

In [ ]:
('AVAX: {:.02f}, USDC: {:.02f}, aAvaUSDC: {:.02f}'
 .format(acc.balance() / (10 ** 18), 
         usdc.balanceOf(acc.address) / (10 ** usdc.decimals()),
         aAvaUSDC.balanceOf(acc.address) / (10 ** aAvaUSDC.decimals())))

### fireblocks integration brainstorm

option 1 - create wrapper/decorator around contract methods when called  
decorator checks whether `acc` is fireblocks type or regular type

In [ ]:
if prod:
    acc = load_fireblocks_account(API_KEY)
else:
    acc = accounts.add()
    
acc.address

def FireblocksTxn(func):
    
    def wrapper(ctc, args):
        
        # are you a fireblocks wallet?
        if fireblocks acc:
            ctc.address
            ctc.abi

            from fireblocksSDK
            fireblocks.call_contract(acc,
                                     contract_address,
                                     abi, 
                                     args(payload))
        else:
            return func(*args)
        

    
FireblocksTxn(aave_pool.withdraw)(usdc.address, 1_000_000, acc.address, 0, {'from': acc})

option 2 - subclass brownie `Contract`  
when any method is called check whether method is a `write` type and `acc` is fireblocks type  
if so use fireblocks sdk

In [ ]:
if prod:
    acc = load_fireblocks_account(API_KEY)
else:
    acc = accounts.add()
    

class FireblocksContract(Contract):
    
    # on any method call check if read only or write type (brownie.network.contract.ContractTx)
    # if write type and acc is a fireblocks account then use fireblocks sdk
    if isinstance(method, brownie.network.contract.ContractTx) and isinstance(acc, FireblocksAcc):
        
        call_fireblocks_sdk(self.address, method_name, acc, *args)
        
    else:
        return method(*args)
        
from brownie import Contract
Contract = FireblocksContract(Contract)
aave_pool = Contract.from_abi('Aave V3 Pool', '0x794a61358d6845594f94dc1db02a252b5b4814ad', abi)
aave_pool.withdraw(usdc.address, 1_000_000, acc.address, 0, {'from': acc})